In [1]:
import pandas as pd
import torch as th
import torchvision as thv
import numpy as np
import matplotlib.pyplot as plt
from torchinfo import summary
from sklearn.preprocessing import OneHotEncoder
import os
from utils import CustomTargetTransformation
import time
SAVE_PATH = r"E:\PyImage_ComputerVision\DeepLearningFrameWorks\Working Data\OutputData\PyTorch\New folder"

In [2]:
DEVICE = "cuda" if th.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [3]:
train_dataset = thv.datasets.CIFAR10(root="E:\PyImage_ComputerVision\DeepLearningFrameWorks\Working Data\InputData\Pre-BuiltIn Datasets",
                                      train=True,
                                      download=True,
                                      transform=thv.transforms.Compose([thv.transforms.ToTensor()]))

test_dataset = thv.datasets.CIFAR10(root="E:\PyImage_ComputerVision\DeepLearningFrameWorks\Working Data\InputData\Pre-BuiltIn Datasets",
                                    train=False,
                                    download=True,
                                    transform=thv.transforms.Compose([thv.transforms.ToTensor()]))

Files already downloaded and verified
Files already downloaded and verified


In [4]:
## Training the One Hot Encoder on the Previous Downloaded Targets

OHE = OneHotEncoder(sparse_output=False) 

OHE.fit(np.array(train_dataset.targets).reshape(-1,1))

OneHotEncoder(sparse_output=False)

In [5]:
from torchvision.transforms import v2

X_Transforms = v2.Compose([
                            v2.ToImage(),
                            v2.Resize(size=(32,32),antialias=True),
                            v2.ToDtype(th.float32,scale=True)
                          ])

Y_Transforms = v2.Compose([
                              CustomTargetTransformation()  
                          ])

In [6]:
OHE_train_dataset = thv.datasets.CIFAR10(root="E:\PyImage_ComputerVision\DeepLearningFrameWorks\Working Data\InputData\Pre-BuiltIn Datasets",
                                         train=True,
                                         download=True,
                                         transform=X_Transforms,
                                         target_transform=Y_Transforms)

OHE_test_dataset = thv.datasets.CIFAR10(root="E:\PyImage_ComputerVision\DeepLearningFrameWorks\Working Data\InputData\Pre-BuiltIn Datasets",
                                        train=False,
                                        download=True,
                                        transform=X_Transforms,
                                        target_transform=Y_Transforms)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
train_dataloader = th.utils.data.DataLoader(OHE_train_dataset,
                                            batch_size=32,
                                            num_workers=8,
                                            shuffle=True)

test_dataloader = th.utils.data.DataLoader(OHE_test_dataset,
                                           shuffle=False,
                                           num_workers=8,
                                           batch_size=32)

In [8]:
OHE_train_dataset[0][0].shape

NameError: name 'OHE' is not defined

In [ ]:
OHE_train_dataset[0]

In [ ]:
model_V0 = th.nn.Sequential(
                            th.nn.Conv2d(in_channels=3,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            th.nn.ReLU(),
                            th.nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            th.nn.ReLU(),
                            th.nn.MaxPool2d(kernel_size=2),
                            th.nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            th.nn.ReLU(),
                            th.nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            th.nn.ReLU(),
                            th.nn.MaxPool2d(kernel_size=2),
                            th.nn.Flatten(),
                            th.nn.Linear(in_features=250,out_features=10)
)

In [ ]:
summary(model_V0,
        input_size=(1,3,32,32),
        verbose=1,
        col_names=["input_size","output_size","num_params","trainable"],
        row_settings=["var_names"],
        col_width=18);

In [9]:
loss_fn = th.nn.CrossEntropyLoss()

optimizer = th.optim.Adam(model.parameters(),
                          lr=0.001)

NameError: name 'model_V0' is not defined

In [14]:
def accuracy(y_pred,Y):
    correct = th.eq(y_pred,Y).sum().item()
    acc = (correct/len(y_pred))*100.0
    return acc

In [ ]:
EPOCHS = 20
TRAIN_DL_LEN = len(train_dataloader)
TEST_DL_LEN = len(test_dataloader)

model_V0.to(DEVICE)
for epoch in range(EPOCHS):
    start = time.time()
    model_V0.train()
    epoch_train_loss = 0.0
    epoch_train_accuracy = 0.0
    for X,Y in train_dataloader:
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)

        Y_logits = model_V0(X).squeeze()
        y_pred = th.argmax(th.softmax(Y_logits,1),1)
        y_true = th.argmax(Y,1)
        loss = loss_fn(Y_logits,Y)
        acc = accuracy(y_pred,y_true)

        epoch_train_accuracy += acc
        epoch_train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model_V0.eval()
    epoch_test_loss = 0.0
    epoch_test_accuracy = 0.0
    
    with th.no_grad():
        for X,Y in test_dataloader:
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)

            Y_logits = model_V0(X).squeeze()

            y_pred = th.argmax(th.softmax(Y_logits,1),1)
            y_true = th.argmax(Y,1)
            loss = loss_fn(Y_logits,Y)
            acc = accuracy(y_pred,y_true)

            epoch_test_accuracy += acc
            epoch_test_loss += loss

    epoch_test_accuracy /= TEST_DL_LEN
    epoch_test_loss /= TEST_DL_LEN
    epoch_train_accuracy /= TRAIN_DL_LEN
    epoch_train_loss /= TRAIN_DL_LEN
    end = time.time()    
    print(f"Epoch {epoch}[{round(end-start,2)}]:- |Train Loss:- {epoch_train_loss:.2f} <--------> Train Accuracy:- {epoch_train_accuracy:.2f}|++++++++|Test Loss:- {epoch_test_loss:.2f} <--------> Test Accuracy :- {epoch_test_accuracy:.2f}|")
    #save_model_weigths(MODEL=model_V0,epoch=epoch,extra_name="model_V0")